[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karnsaurabhkumar/soscipy/blob/master/examples/1_Getting%20Started.ipynb)


# SoSciPy
Soscipy is a python library to simplify working with data specially in social sciences. While there are several packages out there, I have personally found it difficult to find out the right library to stick to and the right recepies to use. Unlike other domain where computational methods have seen a rapid growth, social sciences remain a relatively unexplored area. This is first of the 4 tutorials which will explore data analysis in education. 

There are four parts to soscipy:
- **Data Analysis** : Aims to make rapid analysis easy while not compromising on any functionalities and extendability
- **Data Processing** : Makes common actions with structured data easy and accessible without needing expertiese in computer science
- **Data Visualisation** : Rapid visualisations while ensuring that the output is publication quality
- **Utilities** : A set of utilities that you can plug and play to make your workflow easy

In [14]:
#Installation
!pip install --upgrade soscipy

  Attempting uninstall: soscipy
    Found existing installation: soscipy 0.0.24
    Uninstalling soscipy-0.0.24:
      Successfully uninstalled soscipy-0.0.24


In [22]:
pd.set_option('display.max_rows', None)

In [15]:
import pandas as pd
from soscipy.process import dfops

In [16]:
f1_path = 'https://github.com/karnsaurabhkumar/soscipy/blob/master/data/gyan_data.csv?raw=true'
f2_path = 'https://github.com/karnsaurabhkumar/soscipy/blob/master/data/rangin_justicehub-file.csv?raw=true'

In [17]:
f1 = pd.read_csv(f1_path)
f2 = pd.read_csv(f2_path)

In [18]:
list1 = list(f1.iloc[:,0])
list2 = list(f2.iloc[:,0])

In [19]:
match = dfops.string_matcher(list1,list2)

In [20]:
df = match.get_matched_list()

In [23]:
df[df.similairity < 0.99]

,left_side,right_side,similairity
21,Abdul Nazeer,S. Abdul Nazeer,0.896134
27,Indira Banerjee,Indira Bannerjee,0.866713
44,Bhushan Ramakrishna Gavai,Bhushan RamaKrishna Gavai,0.813308
50,Ajjikuttira Somaiah Bopanna,Ajjikutira Somaiah Bopanah,0.862710
60,Harilal Jekisundas Kania\r\n,Harilal Jekisundas Kania,0.954982
65,Bijan Kumar Mukherjea\r\n,Bijan Kumar Mukherjea,0.926006
67,Sudhi Ranjan Das\r\n,Sudhi Ranjan Das,0.887627
69,Bhuvneshwar Prasad Sinha\n,Bhuvneshwar Prasad Sinha,0.969656
81,P.N. Bhagwati\r\n,P.N. Bhagwati,0.895712
85,Sabyasachi Mukherjee\r\n,Sabyasachi Mukherjee,0.929294
